In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [2]:
import sys
import importlib
from tqdm.notebook import tqdm
import copy

%load_ext autoreload
%autoreload 2

In [3]:
## TODO: avoid putting . in file names in future
## TODO: Include GPU Option

# Load modules
path_to_basic_neural_processing_modules = '/Users/josh/Documents/github_repos'
path_to_GCAMP_ROI = '/Users/josh/Documents/github_repos'
sys.path.append(path_to_basic_neural_processing_modules)
sys.path.append(path_to_GCAMP_ROI)

from simclr_helpers import *

# Load Data
path_to_h5 = f'./data/masks_20211202_balanced.h5' # raw data h5
model_name = 'ResNet18_simCLR_model_202112078_EOD_transfmod=norm' # path to pth file
path_to_classifier = './classifier-models/logreg_model_0.01.pkl' # path to logististic classifier pickle file
path_to_output_file = './latent-outputs/test-run-latents.pkl' # output results file
path_to_model = './simclr-models/' + model_name + '.pth'
h5_keys = ['RHmasks', 'SYTmasks', 'NPmasks'] # list of keys within h5 to concatenate

path_to_architecture = copy.copy(model_name) # path to .py file with architecture

data = load_h5(path_to_h5, h5_keys)
data = drop_nan_imgs(data)
print(data.shape[0], 'examples loaded.')


40353 examples loaded.


In [4]:
# data = data[:2048]
# data = data[:data.shape[0]//100]


In [5]:
# Instantiate Model
model_file = importlib.import_module(path_to_architecture)
model = model_file.get_model(path_to_model)
model.eval();


In [8]:
# Create Data Sets / Data Loaders
dataset, dataloader = model_file.get_dataset_dataloader(data) ## TODO: Troubleshoot the runtime on this

In [9]:
# Get Model Latents
latents = dataloader_to_latents(dataloader, model).numpy()

In [10]:
latents2 = latents.copy()

In [11]:
# Load Logistic Model
classifier_model = load_classifier_model(path_to_classifier)

In [12]:
# Predict ROIs — Save to File
preds = classifier_model.predict_proba(latents)
ret = get_returns(latents, classifier_model, path_to_model, path_to_classifier)
pickle_helpers.simple_save(ret, path_to_output_file)
